# AUSTINSCHOOLS.PY
#### Create the CSV with data from the GreatSchools.org web APIs
#### author : suchita mujumdar

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import xmltodict

# GreatSchools API Key
from keys import api_keys


In [2]:
f = open('output/fieldErrors.txt','w')
def extract(mydict, field):
    try:
        return mydict[field]
    except Exception as e:
        f.write("extract{}: {}".format(type(e), e))
        return np.nan

In [3]:
def extract_ethnicity(ethnicity_data, ethnicity):
    try:
        for x in ethnicity_data:
            if x["name"] == ethnicity:
                return x["value"]
    except Exception as e:
        f.write("extract_ethnicity {}: {}".format(type(e), e))
        return np.nan
    

In [4]:
def get_census_data(gsId):   
    census_url = "https://api.greatschools.org/school/census/TX/{0}?key={1}".format(gsId,api_keys.gs_key)
    census = xmltodict.parse(requests.get(census_url).content)['census-data']
    return census

In [5]:
def get_schools(school_type):
    base_url = "https://api.greatschools.org/schools/TX/Austin/{0}?key={1}&limit=-1&sort=parent_rating".format(school_type, api_keys.gs_key)
    schools = xmltodict.parse(requests.get(base_url).content)['schools']
    df = pd.DataFrame(schools, columns=schools.keys())
    return df


In [6]:
df_public = get_schools("public")
df_private = get_schools("private")
df_charter = get_schools("charter")
df = df_public.append(df_private)
df = df.append(df_charter)

In [7]:
school_data = ['gsId', 'name', 'type','gradeRange','enrollment','gsRating','parentRating','city','state',\
    'districtId','district','districtNCESId','address','phone','fax','website','ncesId','lat',\
    'lon']
for field in school_data:
    df[field] = df.school.map(lambda x: extract(x, field))

In [8]:
df.iloc[0]

school            {'gsId': '527', 'name': 'Blackshear Elementary...
gsId                                                            527
name                                   Blackshear Elementary School
type                                                         public
gradeRange                                                     PK-5
enrollment                                                      302
gsRating                                                          8
parentRating                                                      5
city                                                         Austin
state                                                            TX
districtId                                                      115
district                         Austin Independent School District
districtNCESId                                              4808940
address                  1712 East 11th Street, \nAustin, TX  78702
phone                                           

In [9]:
df.head()

,school,gsId,name,type,gradeRange,enrollment,gsRating,parentRating,city,state,districtId,district,districtNCESId,address,phone,fax,website,ncesId,lat,lon
0,"{'gsId': '527', 'name': 'Blackshear Elementary...",527,Blackshear Elementary School,public,PK-5,302,8,5,Austin,TX,115,Austin Independent School District,4808940,"1712 East 11th Street, \nAustin, TX 78702",(512) 414-2021,(512) 477-7640,http://www.blackshearyellowjackets.org/,480894000300,30.266968,-97.72217
1,"{'gsId': '532', 'name': 'Bryker Woods Elementa...",532,Bryker Woods Elementary School,public,PK-6,400,9,5,Austin,TX,115,Austin Independent School District,4808940,"3309 Kerbey Lane, \nAustin, TX 78703",(512) 414-2054,(512) 459-9047,https://brykerwoodspta.org/,480894000305,30.304869,-97.750626
2,"{'gsId': '535', 'name': 'Casis Elementary Scho...",535,Casis Elementary School,public,PK-5,793,10,5,Austin,TX,115,Austin Independent School District,4808940,"2710 Exposition Boulevard, \nAustin, TX 78703",(512) 414-2062,(512) 477-1776,http://casiselementary.com/,480894000308,30.304344,-97.76505
3,"{'gsId': '557', 'name': 'Maplewood Elementary ...",557,Maplewood Elementary School,public,PK-6,480,4,5,Austin,TX,115,Austin Independent School District,4808940,"3808 Maplewood Avenue, \nAustin, TX 78722",(512) 414-4402,(512) 472-8559,www.maplewoodelementary.com,480894000330,30.293337,-97.714905
4,"{'gsId': '559', 'name': 'Mathews Elementary Sc...",559,Mathews Elementary School,public,PK-6,425,8,5,Austin,TX,115,Austin Independent School District,4808940,"906 West Lynn Street, \nAustin, TX 78703",(512) 414-4406,(512) 476-2108,https://www.mathews360.com/,480894000332,30.278215,-97.760796


In [10]:
df["census"] = df["gsId"].apply(get_census_data)

In [13]:
df['census'][0]

OrderedDict([('schoolName', 'Blackshear Elementary School'),
             ('address', '1712 East 11th Street,  Austin, TX  78702'),
             ('latitude', '30.266968'),
             ('longitude', '-97.72217'),
             ('phone', '(512) 414-2021'),
             ('type', 'public'),
             ('district', 'Austin Independent School District'),
             ('headOfficialName', 'MR RICHARD GARNER'),
             ('enrollment', '302'),
             ('freeAndReducedPriceLunch', '81.1'),
             ('ethnicities',
              OrderedDict([('ethnicity',
                            [OrderedDict([('name', 'White, non-Hispanic'),
                                          ('value', '9.97'),
                                          ('year', '2016')]),
                             OrderedDict([('name', 'Black, non-Hispanic'),
                                          ('value', '27.84'),
                                          ('year', '2016')]),
                             OrderedD

In [12]:
df = df.reset_index()

In [16]:
df.head()

,school,gsId,name,type,gradeRange,enrollment,gsRating,parentRating,city,state,...,district,districtNCESId,address,phone,fax,website,ncesId,lat,lon,census
0,"{'gsId': '527', 'name': 'Blackshear Elementary...",527,Blackshear Elementary School,public,PK-5,302,8,5,Austin,TX,...,Austin Independent School District,4808940,"1712 East 11th Street, \nAustin, TX 78702",(512) 414-2021,(512) 477-7640,http://www.blackshearyellowjackets.org/,480894000300,30.266968,-97.72217,"{'schoolName': 'Blackshear Elementary School',..."
1,"{'gsId': '532', 'name': 'Bryker Woods Elementa...",532,Bryker Woods Elementary School,public,PK-6,400,9,5,Austin,TX,...,Austin Independent School District,4808940,"3309 Kerbey Lane, \nAustin, TX 78703",(512) 414-2054,(512) 459-9047,https://brykerwoodspta.org/,480894000305,30.304869,-97.750626,{'schoolName': 'Bryker Woods Elementary School...
2,"{'gsId': '535', 'name': 'Casis Elementary Scho...",535,Casis Elementary School,public,PK-5,793,10,5,Austin,TX,...,Austin Independent School District,4808940,"2710 Exposition Boulevard, \nAustin, TX 78703",(512) 414-2062,(512) 477-1776,http://casiselementary.com/,480894000308,30.304344,-97.76505,"{'schoolName': 'Casis Elementary School', 'add..."
3,"{'gsId': '557', 'name': 'Maplewood Elementary ...",557,Maplewood Elementary School,public,PK-6,480,4,5,Austin,TX,...,Austin Independent School District,4808940,"3808 Maplewood Avenue, \nAustin, TX 78722",(512) 414-4402,(512) 472-8559,www.maplewoodelementary.com,480894000330,30.293337,-97.714905,"{'schoolName': 'Maplewood Elementary School', ..."
4,"{'gsId': '559', 'name': 'Mathews Elementary Sc...",559,Mathews Elementary School,public,PK-6,425,8,5,Austin,TX,...,Austin Independent School District,4808940,"906 West Lynn Street, \nAustin, TX 78703",(512) 414-4406,(512) 476-2108,https://www.mathews360.com/,480894000332,30.278215,-97.760796,"{'schoolName': 'Mathews Elementary School', 'a..."


In [15]:
del(df['index'])

In [ ]:
df["census"] = df["gsId"].apply(get_census_data)

In [17]:
census_data = ['schoolName', "district",'address', 'latitude','longitude','phone','type','headOfficialName',\
    'enrollment','freeAndReducedPriceLunch','idea','plan504','percentTeachersInFirstSecondYear','ethnicities']
for field in census_data:
    df[field] = df.census.map(lambda x: extract(x, field))

In [18]:
df.head()

,school,gsId,name,type,gradeRange,enrollment,gsRating,parentRating,city,state,...,census,schoolName,latitude,longitude,headOfficialName,freeAndReducedPriceLunch,idea,plan504,percentTeachersInFirstSecondYear,ethnicities
0,"{'gsId': '527', 'name': 'Blackshear Elementary...",527,Blackshear Elementary School,public,PK-5,302,8,5,Austin,TX,...,"{'schoolName': 'Blackshear Elementary School',...",Blackshear Elementary School,30.266968,-97.72217,MR RICHARD GARNER,81.1,8.4,4.2,15,"{'ethnicity': [{'name': 'White, non-Hispanic',..."
1,"{'gsId': '532', 'name': 'Bryker Woods Elementa...",532,Bryker Woods Elementary School,public,PK-6,400,9,5,Austin,TX,...,{'schoolName': 'Bryker Woods Elementary School...,Bryker Woods Elementary School,30.304869,-97.750626,MS KRISTINA MUEHLING,9.75,6.1,4.2,3.8,"{'ethnicity': [{'name': 'White, non-Hispanic',..."
2,"{'gsId': '535', 'name': 'Casis Elementary Scho...",535,Casis Elementary School,public,PK-5,793,10,5,Austin,TX,...,"{'schoolName': 'Casis Elementary School', 'add...",Casis Elementary School,30.304344,-97.76505,MR SAMUEL TINNON,2.14,6.2,6.1,5.9,"{'ethnicity': [{'name': 'White, non-Hispanic',..."
3,"{'gsId': '557', 'name': 'Maplewood Elementary ...",557,Maplewood Elementary School,public,PK-6,480,4,5,Austin,TX,...,"{'schoolName': 'Maplewood Elementary School', ...",Maplewood Elementary School,30.293337,-97.714905,MS VICKIE JACOBSON,43.07,10.5,6,6.7,"{'ethnicity': [{'name': 'White, non-Hispanic',..."
4,"{'gsId': '559', 'name': 'Mathews Elementary Sc...",559,Mathews Elementary School,public,PK-6,425,8,5,Austin,TX,...,"{'schoolName': 'Mathews Elementary School', 'a...",Mathews Elementary School,30.278215,-97.760796,G MARTINO-BREWSTER,36.0,4.4,4.2,3.6,"{'ethnicity': [{'name': 'White, non-Hispanic',..."


In [20]:
df['ethnicity'] = df['ethnicities'].map(lambda x: extract(x, 'ethnicity'))
df.iloc[0]

school                              {'gsId': '527', 'name': 'Blackshear Elementary...
gsId                                                                              527
name                                                     Blackshear Elementary School
type                                                                           public
gradeRange                                                                       PK-5
enrollment                                                                        302
gsRating                                                                            8
parentRating                                                                        5
city                                                                           Austin
state                                                                              TX
districtId                                                                        115
district                                           Aus

In [21]:
ethnicities = ['White, non-Hispanic', 'Black, non-Hispanic', 'Hispanic','Multiracial',\
        'Asian','Native American or Native Alaskan','Native Hawaiian or Other Pacific Islander']
for ethnicity in ethnicities:
    df[ethnicity] = df.ethnicity.map(lambda x: extract_ethnicity(x, ethnicity))


In [22]:
df.iloc[0]

school                                       {'gsId': '527', 'name': 'Blackshear Elementary...
gsId                                                                                       527
name                                                              Blackshear Elementary School
type                                                                                    public
gradeRange                                                                                PK-5
enrollment                                                                                 302
gsRating                                                                                     8
parentRating                                                                                 5
city                                                                                    Austin
state                                                                                       TX
districtId                                        

In [23]:
df.columns

Index(['school', 'gsId', 'name', 'type', 'gradeRange', 'enrollment',
       'gsRating', 'parentRating', 'city', 'state', 'districtId', 'district',
       'districtNCESId', 'address', 'phone', 'fax', 'website', 'ncesId', 'lat',
       'lon', 'census', 'schoolName', 'latitude', 'longitude',
       'headOfficialName', 'freeAndReducedPriceLunch', 'idea', 'plan504',
       'percentTeachersInFirstSecondYear', 'ethnicities', 'ethnicity',
       'White, non-Hispanic', 'Black, non-Hispanic', 'Hispanic', 'Multiracial',
       'Asian', 'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander'],
      dtype='object')

In [24]:
df = df[[ 'gsId', 'name', 'type', 'gradeRange', 'enrollment',
       'gsRating', 'parentRating', 'city', 'state', 'districtId', 'district',
       'districtNCESId', 'address', 'phone', 'fax', 'website', 'ncesId', 'lat',
       'lon', 'schoolName', 'latitude', 'longitude',
       'headOfficialName', 'freeAndReducedPriceLunch', 'idea', 'plan504',
       'percentTeachersInFirstSecondYear', 
       'White, non-Hispanic', 'Black, non-Hispanic', 'Hispanic', 'Multiracial',
       'Asian', 'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander' ]]


In [25]:
df.head()

,gsId,name,type,gradeRange,enrollment,gsRating,parentRating,city,state,districtId,...,idea,plan504,percentTeachersInFirstSecondYear,"White, non-Hispanic","Black, non-Hispanic",Hispanic,Multiracial,Asian,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander
0,527,Blackshear Elementary School,public,PK-5,302,8,5,Austin,TX,115,...,8.4,4.2,15,9.97,27.84,57.39,3.09,0.69,1.03,0.0
1,532,Bryker Woods Elementary School,public,PK-6,400,9,5,Austin,TX,115,...,6.1,4.2,3.8,68.25,1.75,20.75,5.0,4.0,0.25,0.0
2,535,Casis Elementary School,public,PK-5,793,10,5,Austin,TX,115,...,6.2,6.1,5.9,79.32,1.26,10.84,5.17,3.15,0.13,0.13
3,557,Maplewood Elementary School,public,PK-6,480,4,5,Austin,TX,115,...,10.5,6,6.7,38.1,17.97,33.55,8.23,2.16,0.0,0.0
4,559,Mathews Elementary School,public,PK-6,425,8,5,Austin,TX,115,...,4.4,4.2,3.6,44.0,5.18,35.76,5.65,8.94,0.24,0.24


In [26]:
df.to_csv("data/AustinSchools.csv")

In [27]:
!open 'data/AustinSchools.csv'